In [1]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from matplotlib.pyplot import subplots
import joblib

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
root_dir = 'C:/Users/elmow/Documents/Kaggle/Dog_Classification/data/train'
csv_file = 'C:/Users/elmow\Documents/Kaggle/Dog_Classification/data/labels.csv'

In [4]:
#Hyperparameters
TRAIN_SIZE = 0.8
TEST_SIZE = 0.2
BATCH_SIZE = 64

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.data = csv_file
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.loc[idx, "id"] + ".jpg")
        image = Image.open(img_name)
        label_int = torch.tensor(self.data.loc[idx, "breed"], dtype=torch.long)
        # one_hot_label = torch.zeros(self.num_classes)
        # one_hot_label[label_int] = 1
        if self.transform:
            image = self.transform(image)
        return image, label_int

labels_df = pd.read_csv(csv_file)
num_classes = labels_df['breed'].nunique()

# Create a label encoder object
#class_encoder


# Fit and transform the 'breed' column to get integer labels
for i in range(labels_df.shape[0]):
    labels_df.loc[i, 'breed'] = mapping[labels_df.loc[i, 'breed']]

#Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Imagenet normalization
])

#Datasets
dataset = CustomImageDataset(root_dir=root_dir, csv_file=labels_df, transform=transform)
total_samples = len(dataset)
train_size = TRAIN_SIZE
test_size = TEST_SIZE
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)